In [1]:
from time import time
import numpy as np
import tenseal as ts
import torch
import base64
import os

from tqdm import tqdm_notebook

In [2]:
def write_data(file_name, file_content):
    if type(file_content) == bytes:
        file_content = base64.b64encode(file_content)
        
    with open(file_name, 'wb') as f:
        f.write(file_content)

def read_data(file_name):
    with open(file_name, 'rb') as f:
        file_content = f.read()
    
    return base64.b64decode(file_content)

In [3]:
def load_enc_data(read_data):
    context = ts.context_from(read_data("D:/data/server/public_context.txt"))
    
    path_x = "D:/data/server/enc_x/"
    file_list_x = os.listdir(path_x)

    enc_x = []
    for i, file in tqdm_notebook(enumerate(file_list_x)):
        if i == 100:
            break
        temp = read_data(file_name=path_x + file)
        temp = ts.lazy_ckks_vector_from(temp)
        temp.link_context(context)
        enc_x.append(temp)
        
    path_y = "D:/data/server/enc_x/"
    file_list_y = os.listdir(path_y)
    
    enc_y = []
    for i, file in tqdm_notebook(enumerate(file_list_y)):
        if i == 100:
            break
        temp = ts.lazy_ckks_vector_from(read_data(file_name=path_y + file))
        temp.link_context(context)
        enc_y.append(temp)
        
    return context, enc_x, enc_y

In [4]:
context, enc_x, enc_y = load_enc_data(read_data)

C:\Users\dwcho\anaconda3\envs\crypto\lib\site-packages\ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

C:\Users\dwcho\anaconda3\envs\crypto\lib\site-packages\ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

In [5]:
# To get initial weight and bias
# You can also randomly initialize it
# But it is comfortable to use torch to get (weight, bias) pair 

class LR(torch.nn.Module):
    def __init__(self, n_features):
        super(LR, self).__init__()
        self.lr = torch.nn.Linear(n_features, 1)
        
    def forward(self, x):
        out = torch.sigmoid(self.lr(x))
        return out

In [6]:
# Linear Regression Model that supports training
# but several settings have to be set baiscally and tough

class EncryptedLR:
    def __init__(self, torch_lr):
        # we have to change the shape: 1xN -> N
        self.weight = torch_lr.lr.weight.data.tolist()[0]  
        self.bias = torch_lr.lr.bias.data.tolist()
        
        self._delta_w = 0
        self._delta_b = 0
        self._count = 0

    def forward(self, enc_x):
        # forward propagation
        enc_out = enc_x.dot(self.weight) + self.bias
        enc_out = EncryptedLR.sigmoid(enc_out)
        return enc_out
    
    def backward(self, enc_x, enc_out, enc_y):
        # backward propagation
        out_minus_y = (enc_out - enc_y)
        self._delta_w += enc_x * out_minus_y
        self._delta_b += out_minus_y
        self._count += 1
        
    def update_parameters(self):
        # optimizer
        if self._count == 0:
            raise RuntimeError("You should at least run one forward iteration")

        self.weight -= self._delta_w * (1 / self._count) + self.weight * 0.05  
        # "self.weight * 0.05" means l2 regularization
        # it helps keep value between [-5, 5] (sigmoid)
        self.bias -= self._delta_b * (1 / self._count)
        
        self._delta_w = 0
        self._delta_b = 0
        # self._count = 0

    @staticmethod
    def sigmoid(enc_x):
        # sigmoid function approximated between [-5, 5]
        return enc_x.polyval([0.5, 0.197, 0, -0.004])

    def plain_accuracy(self, x_test, y_test):
        # accuracy for regression
        w = torch.tensor(self.weight)
        b = torch.tensor(self.bias)
        out = torch.sigmoid(x_test.matmul(w) + b).reshape(-1, 1)
        correct = torch.abs(y_test - out) < 0.5
        # if the loss under 0.5 -> correct
        return correct.float().mean()

    def encrypt(self, context):
        # encrypt the model before training
        self.weight = ts.ckks_vector(context, self.weight)
        self.bias = ts.ckks_vector(context, self.bias)

    def decrypt(self, key):
        # decrypt the model after training
        self.weight = self.weight.decrypt(key)
        self.bias = self.bias.decrypt(key)

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [7]:
def train(
    model, name: str, enc_x_train, enc_y_train
):
    print(f"Training model: {name}")
    train_start = time()
    for _, (enc_x, enc_y) in tqdm_notebook(enumerate(zip(enc_x_train, enc_y_train))):
        enc_out = model.forward(enc_x)
        model.backward(enc_x, enc_out, enc_y)
            
    model.update_parameters()
    train_end = time()
    print(f"Training time is {train_end - train_start}")
          
    return model

In [8]:
enc_LR = EncryptedLR(LR(enc_x[0].size()))
enc_LR.encrypt(context)

In [9]:
EPOCH = 5

for i in range(1, EPOCH+1):
    print(f"training: {i}")
    enc_LR = train(model=enc_LR, name="Logistic Regression", enc_x_train=enc_x, enc_y_train=enc_y)

training: 1
Training model: Logistic Regression


C:\Users\dwcho\anaconda3\envs\crypto\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

ValueError: scale out of bounds